# 다목적 최적화 알고리즘 성능 비교 실험

In [1]:
# 필수 라이브러리 임포트
import pandas as pd
import numpy as np
from load_data import load_and_process_data, create_nutrient_constraints, load_all_menus
from evaluation_function import calculate_harmony_matrix
import os
import random
from pathlib import Path

from performance_metrics import PerformanceEvaluator
from nsga2_optimizer import NSGA2Optimizer
from nsga3_optimizer import NSGA3Optimizer
from spea2_optimizer import SPEA2Optimizer
from emoea_optimizer import EpsilonMOEAOptimizer

In [4]:
# 데이터 로딩 함수
def load_data(diet_db_path, menu_db_path, ingre_db_path):
    print("데이터 로딩 중...")
    diet_db = load_and_process_data(diet_db_path, menu_db_path, ingre_db_path)
    nutrient_constraints = create_nutrient_constraints()
    harmony_matrix, menus, menu_counts, _ = calculate_harmony_matrix(diet_db)
    all_menus = load_all_menus(menu_db_path, ingre_db_path)
    print(f"✅ 데이터 로딩 완료: {len(all_menus)}개 메뉴\n")
    
    return diet_db, nutrient_constraints, harmony_matrix, all_menus

def compare_optimizers_performance(f, initial_diet_path, diet_db, menu_db_path, ingre_db_path,
                                   all_menus, nutrient_constraints, harmony_matrix,
                                   generations: int = 100, num_runs: int = 10):
    print("초기 식단 로딩 중...")
    weekly_diet = load_and_process_data(diet_db_path=initial_diet_path,
                                       menu_db_path=menu_db_path,
                                       ingre_db_path=ingre_db_path)
    
    print(f"✅ 초기 식단: {len(weekly_diet.meals)}끼 ({len(weekly_diet.meals)//3}일)\n")
    
    # 최적화 알고리즘 초기화
    optimizers = {
        'NSGA-II': NSGA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'NSGA-III': NSGA3Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'SPEA2': SPEA2Optimizer(all_menus, nutrient_constraints, harmony_matrix),
        'ε-MOEA': EpsilonMOEAOptimizer(all_menus, nutrient_constraints, harmony_matrix)
    }
    
    # 실험 정보 출력
    print("\n" + "="*80)
    print("🔬 알고리즘 성능 비교 실험 시작")
    print("="*80)
    print(f"📊 실험 설계:")
    print(f"   - 알고리즘: NSGA-II, NSGA-III, SPEA2, ε-MOEA")
    print(f"   - 반복 횟수: {num_runs}회/알고리즘")
    print(f"   - 세대 수: {generations} generations")
    print(f"   - 총 실행 횟수: {4 * num_runs}회 (4 알고리즘 × {num_runs}회)")
    print(f"\n📈 평가 지표:")
    print(f"   - Hypervolume (수렴 품질)")
    print(f"   - Spacing (분포 균일성)")
    print(f"   - Diversity (해 다양성)")
    print(f"   - Convergence (수렴 속도)")
    print(f"   - Execution Time (실행 시간)")
    print("="*80 + "\n")
    
    # 성능 평가자 초기화
    evaluator = PerformanceEvaluator(diet_db, weekly_diet, optimizers)
    performance_results = {}
    
    # 각 알고리즘을 num_runs회씩 실행하여 성능 측정
    for name in optimizers.keys():
        print(f"\n{'='*80}")
        print(f"🚀 {name} 알고리즘 실행 중 ({num_runs}회 반복)...")
        print(f"{'='*80}")
        
        performance_results[name] = evaluator.run_single_optimizer(
            optimizer_name=name,
            generations=generations,
            num_runs=num_runs
        )
        
        # 중간 결과 요약 출력
        print(f"\n✅ {name} 완료:")
        if performance_results[name]['hypervolume']:
            print(f"   평균 Hypervolume: {np.mean(performance_results[name]['hypervolume']):.4f}")
            print(f"   평균 실행시간: {np.mean(performance_results[name]['execution_time']):.2f}초")
    
    # 결과 저장
    print(f"\n{'='*80}")
    print("💾 결과 저장 중...")
    evaluator.save_combined_results_to_excel(
        performance_results,
        filename=f'optimization_comparison_results_{f}.xlsx'
    )
    
    print("\n" + "="*80)
    print("✅ 실험 완료!")
    print("="*80)
    print(f"📊 결과 파일: optimization_comparison_results.xlsx")
    print(f"\n다음 단계: '실험 끝났어요!' 라고 말씀하시면 자동으로")
    print(f"  1) 6개 그림 생성 (Radar, Hypervolume, Spacing, Diversity, Time, Heatmap)")
    print(f"  2) Results 섹션 초안 작성")
    print(f"  3) GitHub 업로드")
    print(f"가 진행됩니다.")
    print("="*80 + "\n")
    
    return performance_results

def convert_processed_to_weekly(input_file_path, output_file_path):
    df = pd.read_excel(input_file_path)
    date_columns = [col for col in df.columns if col != '구분']
    
    meal_type_map = {
        '조식': '아침',
        '중식': '점심',
        '석식': '저녁'
    }
    
    result_data = []
    
    for day_idx, date_col in enumerate(date_columns, 1):
        for meal_type_kr, meal_type_output in meal_type_map.items():
            meal_rows = df[df['구분'] == meal_type_kr]
            menus = []
            for _, row in meal_rows.iterrows():
                menu_item = row[date_col]
                if pd.notna(menu_item) and str(menu_item).strip():
                    menu_cleaned = str(menu_item).strip()
                    if menu_cleaned:
                        menus.append(menu_cleaned)
            
            result_data.append({
                'Day': day_idx,
                'MealType': meal_type_output,
                'Menus': ', '.join(menus)
            })
    
    result_df = pd.DataFrame(result_data)
    result_df.to_excel(output_file_path, index=False)    
    return result_df

In [5]:
# 실험 실행
# jeongseong 데이터셋 사용
name = 'jeongseong'

diet_db_path = f'../data/sarang_DB/processed_DB/DIET_{name}.xlsx'
menu_db_path = f'../data/sarang_DB/processed_DB/Menu_ingredient_nutrient_{name}.xlsx'
ingre_db_path = f'../data/sarang_DB/processed_DB/Ingredient_Price_{name}.xlsx'

# 데이터 로드
diet_db, nutrient_constraints, harmony_matrix, all_menus = load_data(
    diet_db_path, menu_db_path, ingre_db_path
)

# 초기 식단 경로 (7일 샘플 식단 사용)
#initial_diet_path = '../data/sarang_DB/processed_DB/Weekly_diet_ex.xlsx'
local_path = Path('../data/sarang_DB/raw_DB/jeongseong')
processed_files = list(local_path.glob('Processed_*.xlsx'))

converted_path = Path('../data/sarang_DB/converted_DB/jeongseong')
converted_path.mkdir(parents=True, exist_ok=True)

converted_files = []
for processed_file in processed_files:
    output_filename = processed_file.name.replace('Processed', 'Converted')
    output_path = converted_path / output_filename
    
    if output_path.exists():
        converted_files.append(output_path)
        continue    
    try:
        convert_processed_to_weekly(processed_file, output_path)
        converted_files.append(output_path)
    except Exception as e:
        print(f"   변환 실패 ({processed_file.name}): {e}")

selected_files = random.sample(converted_files, min(10, len(converted_files)))

for f in selected_files:
    initial_diet_path = f    
    try:
        performance_results = compare_optimizers_performance(
            f.stem,  # 파일명 (확장자 제외)
            initial_diet_path,
            diet_db, menu_db_path, ingre_db_path,
            all_menus, nutrient_constraints, harmony_matrix,
            generations=100,  # 세대 수
            num_runs=10       # 반복 횟수
        )
    except Exception as e:
        continue

데이터 로딩 중...
✅ 데이터 로딩 완료: 931개 메뉴

초기 식단 로딩 중...
✅ 초기 식단: 21끼 (7일)


🔬 알고리즘 성능 비교 실험 시작
📊 실험 설계:
   - 알고리즘: NSGA-II, NSGA-III, SPEA2, ε-MOEA
   - 반복 횟수: 10회/알고리즘
   - 세대 수: 100 generations
   - 총 실행 횟수: 40회 (4 알고리즘 × 10회)

📈 평가 지표:
   - Hypervolume (수렴 품질)
   - Spacing (분포 균일성)
   - Diversity (해 다양성)
   - Convergence (수렴 속도)
   - Execution Time (실행 시간)


🚀 NSGA-II 알고리즘 실행 중 (10회 반복)...

Evaluating NSGA-II...
=== Generation 1/100 ===
제약조건 만족 해: 57/150
현재 세대 수집된 해: 35개
총 수집된 좋은 해: 32개
백업 해: 68개
제약조건 해: 57개
Termination condition met at generation 0

=== get_final_solutions 시작 ===
good_solutions_archive: 32개
backup_solutions: 68개
constraint_solutions: 57개
Run 1/10 completed
=== Generation 1/100 ===
제약조건 만족 해: 113/150
현재 세대 수집된 해: 60개
총 수집된 좋은 해: 59개
백업 해: 67개
제약조건 해: 113개
Termination condition met at generation 0

=== get_final_solutions 시작 ===
good_solutions_archive: 59개
backup_solutions: 67개
constraint_solutions: 113개
Run 2/10 completed
=== Generation 1/100 ===
제약조건 만족 해: 82/150
현재 세대 수집된